In [89]:
import pandas as pd 
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

from tqdm import tqdm 
tqdm.pandas()

In [90]:
df = pd.read_csv('anl_set_jun16_w_extra_from_original_df.csv')

In [91]:
df.columns

Index(['Unnamed: 0', 'image_path', 'split', 'class', 'q0', 'q1', 'q2', 'q3',
       'q4', 'q5', 'q6', 'response_0', 'response_1', 'response_2',
       'response_3', 'response_4', 'response_5', 'response_6'],
      dtype='object')

In [92]:
df['gt'] = df['class'].apply(lambda x: 0 if x == 'drivable' else 1)
df['gt'].value_counts()

gt
0    4186
1    1060
Name: count, dtype: int64

In [93]:
def get_sentiment(text):
    blob = TextBlob(text, analyzer=NaiveBayesAnalyzer())
    return blob.sentiment[0]


In [94]:
def quick_sentiment(text):
    # if first word is yes, 1
    # if first word is no, 0
    # else, 0.5
    text = str(text).lower()
    # strip lspace
    text = text.lstrip()
    if text.startswith('yes'):
        return 1
    elif text.startswith('no'):
        return 0
    else:
        return 0

In [95]:
for i in range(0,7):
    df[f'sentiment_{i}'] = df[f'response_{i}'].progress_apply(quick_sentiment)

100%|██████████| 5246/5246 [00:00<00:00, 675114.10it/s]


In [96]:
df.dropna(subset=['response_0'], inplace=True)

In [97]:
# Sample size per class
sample_size_per_class = 500

# Sample 500 from each class
df_class_0 = df[df['gt'] == 0].sample(n=sample_size_per_class, random_state=1, replace=True)
df_class_1 = df[df['gt'] == 1].sample(n=sample_size_per_class, random_state=1, replace=True)

# Concatenate the samples
df = pd.concat([df_class_0, df_class_1])

In [98]:

df['gt'].value_counts()

gt
0    500
1    500
Name: count, dtype: int64

In [99]:
df = df.reset_index()

In [100]:
# get performance for each question
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for i in range(0,7):
    print(f'Question {i}: {df.loc[0, f"q{i}"]}')
    print(f'Precision: {precision_score(df["gt"], df[f"sentiment_{i}"])}')
    print(f'Recall: {recall_score(df["gt"], df[f"sentiment_{i}"])}')
    print(f'F1: {f1_score(df["gt"], df[f"sentiment_{i}"])}')
    print('')

Question 0: Does this image show a flooded street?
Precision: 0.5788849347568209
Recall: 0.976
F1: 0.7267311988086375

Question 1: Does this image show more than a foot of standing water?
Precision: 1.0
Recall: 0.634
F1: 0.7760097919216646

Question 2: Is the street in this image flooded?
Precision: 0.7452229299363057
Recall: 0.936
F1: 0.8297872340425532

Question 3: Could a car drive through the water in this image?
Precision: 0.0
Recall: 0.0
F1: 0.0

Question 4: Does this image show a visible street?
Precision: 0.4959677419354839
Recall: 0.984
F1: 0.6595174262734583

Question 5: Is there any visible street in this image?
Precision: 0.46316964285714285
Recall: 0.83
F1: 0.5945558739255014

Question 6: Is the view from windshield in this image too obstructed?
Precision: 0.47489539748953974
Recall: 0.454
F1: 0.46421267893660534



/share/ju/conda_virtualenvs/cambrian/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [101]:
# questions 4,5,6 act as filters for 0,1,2,3; 
# if 4,5,6 is positive, then 0,1,2,3 should not even be possible to assess 
# if 4,5,6 is negative, then 0,1,2,3 should be assessed

def filter_4_5_6(row):
    if row['sentiment_6'] == 1:
        return 0
    else:
        return row['gt']

In [102]:
for i in range(0,7):
    df[f'filtered_gt_{i}'] = df.apply(filter_4_5_6, axis=1)

for i in range(0,7):
    print(f'Question {i}: {df.loc[0, f"q{i}"]}')
    print(f'Precision: {precision_score(df["filtered_gt_6"], df[f"sentiment_{i}"])}')
    print(f'Recall: {recall_score(df["filtered_gt_6"], df[f"sentiment_{i}"])}')
    print(f'F1: {f1_score(df["filtered_gt_6"], df[f"sentiment_{i}"])}')
    print('')

Question 0: Does this image show a flooded street?
Precision: 0.3190984578884935
Recall: 0.9853479853479854
F1: 0.48207885304659504

Question 1: Does this image show more than a foot of standing water?
Precision: 0.5425867507886435
Recall: 0.63003663003663
F1: 0.5830508474576271

Question 2: Is the street in this image flooded?
Precision: 0.4124203821656051
Recall: 0.9487179487179487
F1: 0.5749167591564928

Question 3: Could a car drive through the water in this image?
Precision: 0.0
Recall: 0.0
F1: 0.0

Question 4: Does this image show a visible street?
Precision: 0.2691532258064516
Recall: 0.978021978021978
F1: 0.42213438735177866

Question 5: Is there any visible street in this image?
Precision: 0.24553571428571427
Recall: 0.8058608058608059
F1: 0.3763900769888794

Question 6: Is the view from windshield in this image too obstructed?
Precision: 0.0
Recall: 0.0
F1: 0.0



/share/ju/conda_virtualenvs/cambrian/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
